# TAYSIR competition - Track 2 Starter Kit

### Welcome!

This is a notebook to show the structure of a code to participate to the competition.

You can also check the baseline notebook (available in the same archive) for more details about the TAYSIR models and how to use them.

## Prepare your environment

In [ ]:
%pip install -q mlflow torch

In [ ]:
import torch
import mlflow
print('PyTorch version :', torch.__version__)
print('MLflow version :', mlflow.__version__)
import sys
print("Your python version:", sys.version)

This notebook was tested with:
* Torch version: 1.11.0+cu102
* MLFlow version: 1.25.1
* Python version: 3.8.10 [GCC 9.4.0]

Python versions starting at 3.7 are supposed to work (but have not been tested).
## Choosing the phase

First you must select one of the phases/datasets we provide

In [ ]:
TRACK = 2 #always for this track
DATASET = 1

In [ ]:
model_name = f"models/2.{DATASET}.taysir.model"

model = mlflow.pytorch.load_model(model_name)
model.eval()

In [ ]:
try:#RNN
    nb_letters = model.input_size -1
    cell_type = model.cell_type

    print("The alphabet contains", nb_letters, "symbols.")
    print("The type of the recurrent cells is", cell_type.__name__)
except:
    nb_letters = model.distilbert.config.vocab_size
    print("The alphabet contains", nb_letters, "symbols.")
    print("The model is a transformer (DistilBertForSequenceClassification)")

## Load the data

The input data is in the following format :

```
[Number of sequences] [Alphabet size]
[Length of sequence] [List of symbols]
[Length of sequence] [List of symbols]
[Length of sequence] [List of symbols]
...
[Length of sequence] [List of symbols]
```

For example the following data :

```
5 10
6 8 6 5 1 6 7 4 9
12 8 6 9 4 6 8 2 1 0 6 5 9
7 8 9 4 3 0 4 9
4 8 0 4 9
8 8 1 5 2 6 0 5 3 9
```

is composed of 5 sequences and has an alphabet size of 10 (so symbols are between 0 and 9) and the first sequence is composed of 6 symbols (8 6 5 1 6 7 4 9), notice that 8 is the start symbol and 9 is the end symbol.

In [ ]:
file = f"datasets/2.{DATASET}.taysir.valid.words"

sequences = []
with open(file) as f:
    f.readline() #Skip first line (number of sequences, alphabet size)
    for line in f:
        line = line.strip()
        seq = line.split(' ')
        seq = [int(i) for i in seq[1:]] #Remove first value (length of sequence) and cast to int
        sequences.append(seq)

The variable *sequences* is thus **a list of lists**

In [ ]:
print('Number of sequences:', len(sequences))
print('10 first sequences:')
for i in range(10):
    print(sequences[i])

## Model extraction

This is where you will extract your simple own model.

# Submission
Once you are satisfied with your model performance, you must write a predict function that takes a sequence as input (list of integers) and returns 0 or 1 (integer type) for the binary classification track and a probability (float type) for the language modeling track.

Your model is **NOT** a parameter of this function. You should NOT take care of MLFlow saving here. 

*For instance, if you want to submit the original model:*

In [ ]:
def predict(sequence):
    """ Define the function that takes a sequence as a list of integers and return the decision of your extracted model. 
    The function does not need your model as argument. See baseline notebooks for examples."""
    
    #For instance, if you want to submit the original model:
    try: #RNN
        value = model.predict(model.one_hot_encode(sequence)) 
        return value
    except: #Transformer
        def make_future_masks(words:torch.Tensor):
            masks = (words != 0)
            b,l = masks.size()
            #x = einops.einsum(masks, masks, "b i, b j -> b i j")
            x = torch.einsum("bi,bj->bij",masks,masks)
            x *= torch.ones(l,l, dtype=torch.bool, device=x.device).tril()
            x += torch.eye(l,dtype=torch.bool, device=x.device)
            return x.type(torch.int8)
        import numpy
        def predict_next_symbols(model, word):
            """
            Args:
                whole word (list): a complete sequence as a list of integers
            Returns:
                the predicted probabilities of the next ids for all prefixes (2-D ndarray)
            """
            word = [ [ a+1 for a in word ] ]
            word = torch.IntTensor(word)
            model.eval()
            with torch.no_grad():
                attention_mask = make_future_masks(word)
                out = model.forward(word, attention_mask=attention_mask)
                out = torch.nn.functional.softmax(out.logits[0], dim=1)
                return out.detach().numpy()[:, 1:] #  the probabilities for padding id (0) are removed
        def predict_transformer(model, word):
            probs = predict_next_symbols(model, word[:-1])
            probas_for_word = [probs[i,a] for i,a in enumerate(word[1:])]
            value = numpy.array(probas_for_word).prod()
            return float(value)
        return predict_transformer(model, sequence)

## Save and submit 
This is the creation of the model needed for the submission to the competition: you just have to run this cell. It will create in your current directory an **archive** that you can then submit on the competition website.

**You should NOT modify this part, just run it**

In [ ]:
from submit_tools import save_function

save_function(predict, alphabet_size=nb_letters, prefix=f'dataset_{TRACK}.{DATASET}_')